<a href="https://colab.research.google.com/github/inokchoi/ComputerVision/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.datasets.cifar10 import load_data
import tensorflow as tf
import os
import numpy as np

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# (X_train, y_train), (X_test, y_test) = load_data()

mnist = input_data.read_data_sets("MNIST_data/", reshape=False, one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Image Shape: (28, 28, 1)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


In [ ]:
def conv2d(input, filters, kernel_size, strides=1, activation=tf.nn.relu, padding='SAME', name='conv'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.conv2d(input, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                         activation=activation, name=name)
    return out

def dense(input, unit, activation=tf.nn.relu, name='dense'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.dense(input, unit, activation=activation, name=name)
    return out

def max_pool(input, k, s, name='pool'):
    out = tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='SAME', name=name)
    return out

In [ ]:
def res_block(x, filter_num, kernel_size, strides, name='res_block', downsample=False):
    with tf.variable_scope(name, reuse=False):
      x_ = x

      if downsample:
        x_ = conv2d(x_, filter_num, kernel_size, [2,2], name=name)
        x = conv2d(x, filter_num, [1,1], [2,2], name='{}_init'.format(name))
      else:
        x_ = conv2d(x_, filter_num, kernel_size, strides, name=name)
      
      out = x + x_
    return out

#resnet-18 layer
def res_net(x):
    conv1 = conv2d(x, 64, [7,7], [2,2], name='conv1')
    
    pool1 = max_pool(conv1, 3, 2, name='pool1')

    res1 = res_block(pool1, 64, [3,3], [1,1], name='res1_1')
    res1 = res_block(res1, 64, [3,3], [1,1], name='res1_2')

    res2 = res_block(res1, 128, [3,3], [1,1], name='res2_1', downsample=True)
    res2 = res_block(res2, 128, [3,3], [1,1], name='res2_2')
    
    res3 = res_block(res2, 256, [3,3], [1,1], name='res3_1', downsample=True)
    res3 = res_block(res3, 256, [3,3], [1,1], name='res3_2')
    
    res4 = res_block(res3, 512, [3,3], [1,1], name='res4_1', downsample=True)
    res4 = res_block(res4, 512, [3,3], [1,1], name='res4_2')

    dense1 = tf.reshape(res4, [-1, tf.size(res4[0])])
    dense2 = dense(dense1, 10, name='dense2')
    
    return dense2

In [ ]:
epochs = 100
learning_rate = 0.00001
batch_size = 64
dropout = 0.8


# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.int32, [None, 10])
# y_one_hot = tf.squeeze(tf.one_hot(y, 10),axis=1)

keep_prob = tf.placeholder(tf.float32, [])  # dropout (keep probability)
x_resize = tf.image.resize_images(x, [64, 64])

# Construct model
pred = res_net(x)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
res1_1
Tensor("res1_1/add:0", shape=(?, 7, 7, 64), dtype=float32)
res1_2
Tensor("res1_2/add:0", shape=(?, 7, 7, 64), dtype=float32)
res2_1
Tensor("res2_1/add:0", shape=(?, 4, 4, 128), dtype=float32)
res2_2
Tensor("res2_2/add:0", shape=(?, 4, 4, 128), dtype=float32)
res3_1
Tensor("res3_1/add:0", shape=(?, 2, 2, 256), dtype=float32)
res3_2
Tensor("res3_2/add:0", shape=(?, 2, 2, 256), dtype=float32)
res4_1
Tensor("res4_1/add:0", shape=(?, 1, 1, 512), dtype=float32)
res4_2
Tensor("res4_2/add:0", shape=(?, 1, 1, 512), dtype=float32)
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                      gpu_options=tf.GPUOptions(allow_growth=True, visible_device_list='0'))) as sess:
    sess.run(init)
    # Keep training until reach max iterations
    print('Training...')
    for i in range(epochs):
        s = np.random.permutation(len(X_train))
        X_train = X_train[s]
        y_train = y_train[s]
        loss_total, acc_total = 0, 0
        for offset in range(0, len(X_train), batch_size):
            end = offset + batch_size
            batch_xs, batch_ys = X_train[offset:end], y_train[offset:end]

            # Fit training using batch data
            _, acc, loss = sess.run([optimizer, accuracy, cost], feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})

            # Calculate batch accuracy
            # Calculate batch loss
            acc_total += acc
            loss_total += loss

        acc_total /= (len(X_train) / batch_size)
        loss_total /= (len(X_train) / batch_size)
        print("epoch: " + str(i) + ", Training Loss= " + "{:.4f}".format(
            loss_total) + ", Training Accuracy= " + "{:.4f}".format(acc_total))

    print("Optimization Finished!")
    saver.save(sess, './resnet')


Training...
epoch: 0, Training Loss= 1.2253, Training Accuracy= 0.6531
epoch: 1, Training Loss= 0.6879, Training Accuracy= 0.7599
epoch: 2, Training Loss= 0.5994, Training Accuracy= 0.7742
epoch: 3, Training Loss= 0.5641, Training Accuracy= 0.7812
epoch: 4, Training Loss= 0.5455, Training Accuracy= 0.7848
epoch: 5, Training Loss= 0.5327, Training Accuracy= 0.7874
epoch: 6, Training Loss= 0.5236, Training Accuracy= 0.7894
epoch: 7, Training Loss= 0.5160, Training Accuracy= 0.7911
epoch: 8, Training Loss= 0.5128, Training Accuracy= 0.7918
epoch: 9, Training Loss= 0.5061, Training Accuracy= 0.7939
epoch: 10, Training Loss= 0.5025, Training Accuracy= 0.7944
epoch: 11, Training Loss= 0.4990, Training Accuracy= 0.7954
epoch: 12, Training Loss= 0.4965, Training Accuracy= 0.7957
epoch: 13, Training Loss= 0.4933, Training Accuracy= 0.7969
epoch: 14, Training Loss= 0.4905, Training Accuracy= 0.7973
epoch: 15, Training Loss= 0.4893, Training Accuracy= 0.7976
epoch: 16, Training Loss= 0.3243, Trai